# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import os
import sys
import time
import json
import requests
import schedule

from typing import Any
from bs4 import BeautifulSoup

sys.path.append(os.path.abspath(".."))
import src.constants as const

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(session: requests.Session, book_url: str) -> dict:
    """Извлекает полную информацию о книге с её страницы.

    Args:
        session (Session): Сессия для HTTP-запросов.
        book_url (str): URL страницы книги.

    Raises:
        ValueError: Если не найдена основная информация о книге.

    Returns:
        dict[str, Any]: Словарь с полной информацией о книге.
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    book_data = {}
    response = session.get(book_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # основная информация
    main_data = soup.find(class_="col-sm-6 product_main")

    if not main_data:
        raise ValueError("Не найдена основная информация о книге")

    title = main_data.find("h1")
    book_data["title"] = (
        title.get_text(strip=True) if title else const.EMPTY_DATA
    )

    price = main_data.find(class_="price_color")
    if price:
        price = price.text.replace("Â", "")
        book_data["price"] = price
    else:
        book_data["price"] = const.EMPTY_DATA

    available = main_data.find(class_="instock availability")
    if available:
        available_data = available.get_text(strip=True)
        available = int("".join(i for i in available_data if i.isdigit()))
        book_data["available"] = available
    else:
        book_data["available"] = const.EMPTY_DATA

    rating_class = main_data.find(class_="star-rating")
    _, rating = rating_class["class"]
    book_data["rating"] = (
        const.RATING_MAP.get(rating, const.UNKNOWN_RATING)
        if rating
        else const.EMPTY_DATA
    )

    # описание
    sub_header = soup.find(id="product_description")
    if not sub_header:
        book_data["description"] = const.EMPTY_DATA
    else:
        description = sub_header.find_next("p")
        book_data["description"] = (
            description.get_text(strip=True)
            if description
            else const.EMPTY_DATA
        )

    # таблица
    product_table = soup.find(class_="table table-striped")
    rows = product_table.select("tr")
    book_table_info = {}
    for row in rows:
        key = row.select_one("th").get_text(strip=True)
        value = row.select_one("td").get_text(strip=True)

        if "Price" in key or "Tax" in key:
            value = value.replace("Â", "")

        book_table_info[key] = value

    book_data["product_info"] = book_table_info

    return book_data

In [4]:
# Используйте для самопроверки
book_url = (
    "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
)
with requests.Session() as session:
    display(get_book_data(session, book_url))

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'available': 22,
 'rating': '3',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = False) -> list[dict[str, Any]]:
    """Парсит данные о всех книгах из каталога.

    Обходит все страницы каталога, извлекает информацию о каждой книге
    и возвращает список с данными. Может сохранять результаты в файл.

    Args:
        is_save (bool, optional): Сохранять ли данные в файл.
            Значение по умолчанию - False.

    Returns:
        list[dict[str, Any]]: Список словарей с данными о книгах.
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    with requests.Session() as session:
        response = session.get(const.START_CATALOGUE_PAGE_URL)
        soup = BeautifulSoup(response.text, "html.parser")

        scraped_books = []
        while True:
            books_redirections = [
                a.get("href", const.LINK_NOT_FOUND)
                for a in soup.select("section ol.row div.image_container a")
            ]
            for book_redirect in books_redirections:
                book_url = const.BASE_URL + book_redirect
                scraped_book = get_book_data(session, book_url)
                scraped_books.append(scraped_book)

            next = soup.select_one("li.next a")
            next_page = (
                next.get("href", const.LINK_NOT_FOUND) if next else None
            )
            if not next_page:
                break

            next_page_url = const.BASE_URL + next_page
            response = session.get(next_page_url)
            soup = BeautifulSoup(response.text, "html.parser")

    if is_save:
        with open("books_data.txt", mode="w", encoding="utf-8") as write:
            json.dump(scraped_books, write, ensure_ascii=False, indent=2)

    return scraped_books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True)  # Допишите ваши аргументы
print(*res[:3], sep="\n")  # демонстрация первых трех результатов
print(type(res), len(res))  # и проверки

{'title': 'A Light in the Attic', 'price': '£51.77', 'available': 22, 'rating': '3', 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up t

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at(const.TASK_START_TIME).do(scrape_books, is_save=True)

try:
    while True:
        schedule.run_pending()
        next_run = schedule.idle_seconds()
        time.sleep(min(next_run, const.DELAY))
except KeyboardInterrupt:
    print("Ручная остановка работы программы")

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [7]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest ../tests/test_scraper.py

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.4.2, pluggy-1.6.0 -- /home/alex/Develop/books_scraper/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/alex/Develop/books_scraper
configfile: pytest.ini
collected 6 items                                                              

../tests/test_scraper.py::TestScraper::test_example_book_page_correct_response PASSED [ 16%]
../tests/test_scraper.py::TestScraper::test_example_book_page_not_found_response PASSED [ 33%]
../tests/test_scraper.py::TestScraper::test_get_correct_book_title PASSED [ 50%]
../tests/test_scraper.py::TestScraper::test_empty_book_title PASSED      [ 66%]
../tests/test_scraper.py::TestScraper::test_get_book_data_success PASSED [ 83%]
../tests/test_scraper.py::TestScraper::test_get_all_books_data_success PASSED [100%]

============================== 6 passed in 1.76s ===============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```